In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 5.1 MB/s 
     |████████████████████████████████| 163 kB 53.5 MB/s 
     |████████████████████████████████| 212 kB 38.5 MB/s 
     |████████████████████████████████| 115 kB 8.6 MB/s 
     |████████████████████████████████| 127 kB 45.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import LSTM, GRU, SimpleRNN, Dense, Embedding
from keras.models import Sequential
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [ ]:
dataset = load_dataset("SalehAhmad/Spam-Ham")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/SalehAhmad___csv/SalehAhmad--Spam-Ham-3b83475258cf549a/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
X = dataset['train']['sms_text']
Y = dataset['train']['label']

In [ ]:
texts = []
labels = []
for i, label in enumerate(dataset['train']['label']):
    texts.append(dataset['train']['sms_text'][i])
    if label == 'ham':
        labels.append(0)
    else:
        labels.append(1)

texts = np.asarray(texts)
labels = np.asarray(labels)


print("number of texts :" , len(texts))
print("number of labels: ", len(labels))

number of texts : 5566
number of labels:  5566


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=1)

In [ ]:
# number of words used as features
max_features = 10000
# cut off the words after seeing 500 words in each document(email)
maxlen = 500
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))
data_train = pad_sequences(sequences_train, maxlen=maxlen)
print(data_train.shape)

Found 7369 unique words: 
(3896, 500)


In [ ]:
#SimpleRNN

model = Sequential()
model.add(Embedding(max_features, 64))
model.add(SimpleRNN(64))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_rnn = model.fit(data_train, y_train, epochs=20, batch_size=60)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          640000    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 648,321
Trainable params: 648,321
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
65/65 [==============================] - 11s 156ms/step - loss: 0.4183 - acc: 0.8024
Epoch 2/20
65/65 [==============================] - 10s 155ms/step - loss: 0.1319 - acc: 0.9605
Epoch 3/20
65/65 [==============================] - 10s 158ms/step - loss: 0.1506 - acc: 0.9363
Epoch 4/20
65/65 [======

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# number of words used as features
max_features = 10000
# cut off the words after seeing 500 words in each document(email)
maxlen = 500
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_test)
sequences_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index
print("Found {0} unique words: ".format(len(word_index)))
data_test = pad_sequences(sequences_test, maxlen=maxlen)
print(data_test.shape)

Found 4655 unique words: 
(1670, 500)


In [ ]:
pred = model.predict(data_test)
pred_label =[]
for i in pred:
  if i >0.5:
    pred_label.append(1)
  else:
    pred_label.append(0)
print(confusion_matrix(pred_label, y_test))
print('Precision for SimpleRNN: %.4f'% precision_score(y_test, pred_label,average='macro'))
print('Recall for SimpleRNN: %.4f' % recall_score(y_test, pred_label, average='macro'))

[[1352  183]
 [ 101   34]]
Precision for SimpleRNN: 0.5663
Recall for SimpleRNN: 0.5436


In [ ]:
# LSTM

modelLSTM = Sequential()
modelLSTM.add(Embedding(max_features, 64))
modelLSTM.add(LSTM(64))
modelLSTM.add(Dense(1, activation='sigmoid'))
modelLSTM.summary()
modelLSTM.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_ltsm = modelLSTM.fit(data_train, y_train, epochs=10, batch_size=60)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 64)          640000    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 673,089
Trainable params: 673,089
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
65/65 [==============================] - 27s 381ms/step - loss: 0.2164 - acc: 0.9274
Epoch 2/10
65/65 [==============================] - 25s 381ms/step - loss: 0.0554 - acc: 0.9861
Epoch 3/10
65/65 [==============================] - 25s 382ms/step - loss: 0.0309 - acc: 0.9923
Epoch 4/10
65/65 [======

In [ ]:
predLSTM = modelLSTM.predict(data_test)
predLSTM_label =[]
for i in predLSTM:
  if i >0.5:
    predLSTM_label.append(1)
  else:
    predLSTM_label.append(0)
print(confusion_matrix(predLSTM_label, y_test))
print('Precision for LSTM: %.4f'% precision_score(y_test, predLSTM_label,average='macro'))
print('Recall for LSTM: %.4f' % recall_score(y_test, predLSTM_label, average='macro'))

[[1372  196]
 [  81   21]]
Precision for LSTM: 0.5404
Recall for LSTM: 0.5205


In [ ]:
#GRU

modelGRU = Sequential()
modelGRU.add(Embedding(max_features, 64))
modelGRU.add(GRU(64))
modelGRU.add(Dense(1, activation='sigmoid'))
modelGRU.summary()
modelGRU.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history_gru = modelGRU.fit(data_train, y_train, epochs=10, batch_size=60)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 64)          640000    
                                                                 
 gru (GRU)                   (None, 64)                24960     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 665,025
Trainable params: 665,025
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
65/65 [==============================] - 24s 342ms/step - loss: 0.2230 - acc: 0.9191
Epoch 2/10
65/65 [==============================] - 22s 343ms/step - loss: 0.0454 - acc: 0.9877
Epoch 3/10
65/65 [==============================] - 22s 343ms/step - loss: 0.0242 - acc: 0.9931
Epoch 4/10
65/65 [======

In [ ]:
predGRU = modelGRU.predict(data_test)
predGRU_label =[]
for i in predGRU:
  if i >0.5:
    predGRU_label.append(1)
  else:
    predGRU_label.append(0)

print(confusion_matrix(predGRU_label, y_test))
print('Precision for LSTM: %.4f'% precision_score(y_test, predGRU_label,average='macro'))
print('Recall for LSTM: %.4f' % recall_score(y_test, predGRU_label, average='macro'))

[[1307  182]
 [ 146   35]]
Precision for LSTM: 0.5356
Recall for LSTM: 0.5304


In [ ]:
wordsCount=[]
for i in X_test:
  noOfWords = len(i.split())
  wordsCount.append(noOfWords)

print(len(wordsCount))
wordsCount.sort()
wordsCount[1669]

1670


96

In [ ]:
small_input = []
medium_input =[]
large_input=[]

small_input_label = []
medium_input_label =[]
large_input_label=[]

for i,c in enumerate(X_test):
  noOfWords = len(c.split())
  if noOfWords <= 32:
    small_input.append(c)
    small_input_label.append(y_test[i])
  elif noOfWords >32 and noOfWords <=64:
    medium_input.append(c)
    medium_input_label.append(y_test[i])
  elif noOfWords > 64:
    large_input.append(c)
    large_input_label.append(y_test[i])

print(len(small_input))
print(len(small_input_label))
print("----------------")
print(len(medium_input))
print(len(medium_input_label))
print("----------------")
print(len(large_input))
print(len(large_input_label))


1586
1586
----------------
77
77
----------------
7
7


In [ ]:
small_input_label = np.array(small_input_label)
medium_input_label = np.array(medium_input_label)
large_input_label = np.array(large_input_label)


In [ ]:
#Short input 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(small_input)
sequences_short = tokenizer.texts_to_sequences(small_input)
data_short = pad_sequences(sequences_short, maxlen=maxlen)


pred_short = model.predict(data_short)
pred_short_label =[]
for i in pred_short:
  if i >0.5:
    pred_short_label.append(1)
  else:
    pred_short_label.append(0)

print(confusion_matrix(pred_short_label, small_input_label))
print('Precision for SimpleRNN: %.4f'% precision_score(small_input_label, pred_short_label,average='macro'))
print('Recall for SimpleRNN: %.4f' % recall_score(small_input_label, pred_short_label, average='macro'))

pred_shortLSTM = modelLSTM.predict(data_short)
pred_shortLSTM_label =[]
for i in pred_shortLSTM:
  if i >0.5:
    pred_shortLSTM_label.append(1)
  else:
    pred_shortLSTM_label.append(0)

print(confusion_matrix(pred_shortLSTM_label, small_input_label))
print('Precision for LSTM: %.4f'% precision_score(small_input_label, pred_shortLSTM_label,average='macro'))
print('Recall for LSTM: %.4f' % recall_score(small_input_label, pred_shortLSTM_label, average='macro'))

pred_shortGRU = modelGRU.predict(data_short)
pred_shortGRU_label =[]
for i in pred_shortGRU:
  if i >0.5:
    pred_shortGRU_label.append(1)
  else:
    pred_shortGRU_label.append(0)

print(confusion_matrix(pred_shortGRU_label, small_input_label))
print('Precision for GRU: %.4f'% precision_score(small_input_label, pred_shortGRU_label,average='macro'))
print('Recall for GRU: %.4f' % recall_score(small_input_label, pred_shortGRU_label, average='macro'))

[[1287  186]
 [  87   26]]
Precision for SimpleRNN: 0.5519
Recall for SimpleRNN: 0.5297
[[1294  198]
 [  80   14]]
Precision for LSTM: 0.5081
Recall for LSTM: 0.5039
[[1236  169]
 [ 138   43]]
Precision for GRU: 0.5586
Recall for GRU: 0.5512


In [ ]:
#Medium input 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(medium_input)
sequences_medium = tokenizer.texts_to_sequences(medium_input)
data_medium = pad_sequences(sequences_medium, maxlen=maxlen)


pred_medium = model.predict(data_medium)
pred_medium_label =[]
for i in pred_medium:
  if i >0.5:
    pred_medium_label.append(1)
  else:
    pred_medium_label.append(0)

print(confusion_matrix(pred_medium_label, medium_input_label))
print('Precision for SimpleRNN: %.4f'% precision_score(medium_input_label, pred_medium_label,average='macro'))
print('Recall for SimpleRNN: %.4f' % recall_score(medium_input_label, pred_medium_label, average='macro'))

pred_mediumLSTM = modelLSTM.predict(data_medium)
pred_mediumLSTM_label =[]
for i in pred_mediumLSTM:
  if i >0.5:
    pred_mediumLSTM_label.append(1)
  else:
    pred_mediumLSTM_label.append(0)

print(confusion_matrix(pred_mediumLSTM_label, medium_input_label))
print('Precision for LSTM: %.4f'% precision_score(medium_input_label, pred_mediumLSTM_label,average='macro'))
print('Recall for LSTM: %.4f' % recall_score(medium_input_label, pred_mediumLSTM_label, average='macro'))

pred_mediumGRU = modelGRU.predict(data_medium)
pred_mediumGRU_label =[]
for i in pred_mediumGRU:
  if i >0.5:
    pred_mediumGRU_label.append(1)
  else:
    pred_mediumGRU_label.append(0)

print(confusion_matrix(pred_mediumGRU_label, medium_input_label))
print('Precision for GRU: %.4f'% precision_score(medium_input_label, pred_mediumGRU_label,average='macro'))
print('Recall for GRU: %.4f' % recall_score(medium_input_label, pred_mediumGRU_label, average='macro'))

[[67  4]
 [ 5  1]]
Precision for SimpleRNN: 0.5552
Recall for SimpleRNN: 0.5653
[[70  5]
 [ 2  0]]
Precision for LSTM: 0.4667
Recall for LSTM: 0.4861
[[61  5]
 [11  0]]
Precision for GRU: 0.4621
Recall for GRU: 0.4236


In [ ]:
#Lerge input 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(large_input)
sequences_large = tokenizer.texts_to_sequences(large_input)
data_large = pad_sequences(sequences_large, maxlen=maxlen)


pred_large = model.predict(data_large)
pred_large_label =[]
for i in pred_large:
  if i >0.5:
    pred_large_label.append(1)
  else:
    pred_large_label.append(0)

print(confusion_matrix(pred_large_label, large_input_label))
print('Precision for SimpleRNN: %.4f'% precision_score(large_input_label, pred_large_label,average='macro'))
print('Recall for SimpleRNN: %.4f' % recall_score(large_input_label, pred_large_label, average='macro'))

pred_largeLSTM = modelLSTM.predict(data_large)
pred_largeLSTM_label =[]
for i in pred_largeLSTM:
  if i >0.5:
    pred_largeLSTM_label.append(1)
  else:
    pred_largeLSTM_label.append(0)

print(confusion_matrix(pred_largeLSTM_label, large_input_label))
print('Precision for LSTM: %.4f'% precision_score(large_input_label, pred_largeLSTM_label,average='macro'))
print('Recall for LSTM: %.4f' % recall_score(large_input_label, pred_largeLSTM_label, average='macro'))

pred_largeGRU = modelGRU.predict(data_large)
pred_largeGRU_label =[]
for i in pred_largeGRU:
  if i >0.5:
    pred_largeGRU_label.append(1)
  else:
    pred_largeGRU_label.append(0)

print(confusion_matrix(pred_largeGRU_label, large_input_label))
print('Precision for GRU: %.4f'% precision_score(large_input_label, pred_largeGRU_label,average='macro'))
print('Recall for GRU: %.4f' % recall_score(large_input_label, pred_largeGRU_label, average='macro'))

[[5 0]
 [2 0]]
Precision for SimpleRNN: 0.5000
Recall for SimpleRNN: 0.3571
[[6 0]
 [1 0]]
Precision for LSTM: 0.5000
Recall for LSTM: 0.4286
[[5 0]
 [2 0]]
Precision for GRU: 0.5000
Recall for GRU: 0.3571


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
